In [2]:
from pynq import Overlay
from time import sleep
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [6]:
class TestChip(Overlay):
    def __init__(self, ol_path, **kwargs):
        super().__init__(ol_path)
        self.heater_base_address = 0x00000000
        self.counter_base_address = 0x00000000
        self.temp_sensor_address = 0x200
        self.counter_address_increament = 0x04
        self.num_oscillators = 4
        for key, value in kwargs.items():
            if "heater_base_address" in key:
                self.heater_base_address = value
            if "counter_base_address" in key:
                self.counter_base_address = value
            if "num_oscillators" in key:
                self.num_oscillators = value
            if "counter_address_increament" in key:
                self.counter_address_increament = value
            if "temp_sensor_address" in key:
                self.temp_sensor_address = value
        self._a = 5 
        self._b = 50
    def XADC_temp(self):
        return ((self.Temp_sensor.read(self.temp_sensor_address) >> 4) * 503.975/4096 - 273.15)
    
    def freq2temp(self, Δf):
        """The values of a and b are for 5 stage ROs in ZYNQ 7000"""
        return Δf * self._a + self._b
    
    def record(self, step_size, number_of_samples, output_name):
        """Records the ROs and the XADC temperature
        
        Parameters:
        step_size (float): seconds
        number_of_samples (int): total number of samples to be recorded 
        output_name (str): outputfile_name
        
        Returns: None
        """
        Output = np.zeros((number_of_samples, self.num_oscillators + 1))
        for i in range(number_of_samples):
            Output[i,self.num_oscillators] = self.XADC_temp()
            for j in range(self.num_oscillators):
                Output[i,j] = ol.AXI_counter_0.read(self.counter_base_address + j * self.counter_address_increament)
        with open(output_name+f'_numsteps_{number_of_samples}_stepsize_{step_size}.npy', 'wb') as f:
            np.save(f, Output)
        return Output
            
    def top_region_heat_on(self, intensity):
        """Turns the top region heat on
        top region heat, heats up the whole top region of the chip
        
        Parameters:
        intensity (int): intensity of the heat (the final temperature depends on ventilation and/or isolation of the chip)
        intensity has to be between 0 to 4
        
        Returns: None
        """
        intensity_list = [0x00000000, 0x00000001, 0x00000003, 0x00000007, 0x0000000F]
        assert intensity < len(intensity_list)
        
        self.AXI4_heater_0.write(self.heater_base_address, intensity_list[intensity])
    def top_region_heat_off(self):
        """Turns the top region heat off
        top region heat, heats up the whole top region of the chip
        
        Returns: None
        """
        self.AXI4_heater_0.write(self.heater_base_address, 0x00000000)
    def __str__(self):
        return f"Number of ROs: {self.num_oscillators}; Current temperature: {self.XADC_temp()}"

In [4]:
ol = TestChip('/home/xilinx/pynq/overlays/RO/RO.bit')

In [5]:
for i in range(5):
    ol.top_region_heat_on(i)
    _ = ol.record(0.1, 100, f'intensity_{i}')
    print(ol)
ol.top_region_heat_off()

Number of ROs: 4; Current temperature: 47.24816894531256
Number of ROs: 4; Current temperature: 55.122778320312534
Number of ROs: 4; Current temperature: 65.7042846679688
Number of ROs: 4; Current temperature: 78.99268798828126
Number of ROs: 4; Current temperature: 89.45115356445314


The register addresses are: 0X00, 0X04, 0X08, 0X0C